In [1]:
import pandas as pd
import datetime

In [14]:
# Create your own path directory txt file

with open('path.txt') as f:
    lines = f.read()
dirs = lines.splitlines()

ledger_path = dirs[0]
inc_path = dirs[1]
bal_path = dirs[2]

In [18]:
# Create New Files

# income statement
pd.DataFrame({'USD':[0.00, 0.00, 0.00, 0.00, 0.00, 0.00]},
             index=['Gross Income', 'Gross Expense',
                    'Operating Income',
                    'Capital Income', 'Captial Expense',
                    'Net Income']).to_csv(inc_path)

# balance sheet
pd.DataFrame({'USD':[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]},
             index=['Net Income A',
                    'Cash', 'Account Receivable', 'Securities', 'Property', 'Total Asset',
                    'Account Payable', 'Unearned Revenue', 'Debt', 'Total Liability',
                    'Contributed Capital', 'Net Income E', 'Retained Earning', 'Total Equity']).to_csv(bal_path)

# ledger
pd.DataFrame(columns=['Date', 'Property', 'Unit', 'Category', 'Description', 'Cost', 'MSRP', 'Bill']).to_csv(ledger_path)

In [15]:
class Statement():
    """
    baseline statement cls
    """
    def __init__(self, path):
        self.df = pd.read_csv(path, index_col=0)
    
    def entry(self, category, amount):
        self.df.at[category, 'USD'] += amount
        self.update()

    def remove(self, category, amount):
        self.df.at[category, 'USD'] -= amount
        self.update()

    def update(self):
        pass

In [5]:
class Inc_stat(Statement):
    """
    Gross Income: rent, late fee
    Gross Expense: pm, repair, maintenance, utilty, legal&pro, insurance, tax
    *Operating Income = [sum]
    Capital Income: invest
    Capital Expense: interest
    *Net Income = [op inc + sum]

    * auto-calculate
    """
    def update(self):
        self.df.at['Operating Income', 'USD'] = self.df.at['Gross Income', 'USD'] - self.df.at['Gross Expense', 'USD']
        self.df.at['Net Income', 'USD'] = self.df.at['Operating Income', 'USD'] + self.df.at['Capital Income', 'USD'] - self.df.at['Captial Expense', 'USD']

inc_stat = pd.DataFrame({'USD':[0.00, 0.00, 0.00, 0.00, 0.00, 0.00]},
                        index=['Gross Income', 'Gross Expense', 
                               'Operating Income', 
                               'Capital Income', 'Captial Expense', 
                               'Net Income'])

inc_stat.to_csv(inc_path)

# inc_stat = Inc_stat(f"/content/{'PRE_income_statement'}.csv")
# inc_stat.entry("Gross Income", 15)
# inc_stat.remove("Gross Expense", 5)
# inc_stat.df

In [57]:
class Bal_sht(Statement):
    """
    *Net Income A: carry from income statement
    Cash: 
    Account Receivable: owed
    Securities: owned shares
    Property: property
    *Total Asset = [ass curr + ass long]
    Account Payable: 
    Unearned Revenue: write offs
    Debt:
    *Total Liability = [liab curr + liab long]
    Contributed Capital
    *Net Income E: carry from income statement
    Retained Earning + [net inc]
    *Total Equity

    * auto-calculate

    Balanced: bool
    *Check eq = [cap + retained earning]
    *Check eq = [ass - liab]
    """
    def update(self):
        self.df.at['Total Asset', 'USD'] = self.df.at['Net Income A', 'USD'] + self.df.at['Cash', 'USD'] + self.df.at['Account Receivable', 'USD'] + self.df.at['Securities', 'USD'] + self.df.at['Property', 'USD']
        self.df.at['Total Liability', 'USD'] = self.df.at['Account Payable', 'USD'] + self.df.at['Unearned Revenue', 'USD'] + self.df.at['Debt', 'USD']
        self.df.at['Total Equity', 'USD'] = self.df.at['Contributed Capital', 'USD'] + self.df.at['Retained Earning', 'USD'] + self.df.at['Net Income E', 'USD']
        if (self.df.at['Total Asset', 'USD'] - self.df.at['Total Liability', 'USD']) == self.df.at['Total Equity', 'USD']:
            self.balance = True
        else:
            self.balance = False


bal_sht = pd.DataFrame({'USD':[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]}, 
                       index=['Net Income A',
                              'Cash', 'Account Receivable', 'Securities', 'Property', 'Total Asset', 
                              'Account Payable', 'Unearned Revenue', 'Debt', 'Total Liability', 
                              'Contributed Capital', 'Net Income E', 'Retained Earning', 'Total Equity'])

bal_sht.to_csv(f"/content/{'PRE_balance_sheet'}.csv")

# bal_sht = Bal_sht(f"/content/{'PRE_balance_sheet'}.csv")
# bal_sht.entry("Asset Current", 15)
# # bal_sht.remove("Gross Expense", 5)
# bal_sht.update()
# bal_sht.df
# bal_sht.balance

In [41]:
class Ledger():
    def __init__(self, path):
        self.df = pd.read_csv(path, index_col=0)

    def entry(self, transact):
        self.df = self.df.append(transact, ignore_index=True)

ledger = pd.DataFrame(columns=['Date', 'Property', 'Unit', 'Category', 'Description', 'Cost', 'MSRP', 'Bill'])
ledger.to_csv(f"/content/{'PRE_general_ledger'}.csv")

# ledger = Ledger(f"/content/PRE_general_ledger.csv")
# t1 = transaction('WF', 4, 'Gross Income', 100.00, 120.00, 'BREI', date=datetime.datetime(2022,1,11).date(), description='WF1 Rent')

# ledger.entry(t1)
# ledger.df

In [63]:
class FinStat():
    """
    For PRE
    """
    def __init__(self, dir):
        self.ledger = Ledger(f"{dir}/PRE_general_ledger.csv")
        self.inc_stat = Inc_stat(f"{dir}/PRE_income_statement.csv")
        self.bal_sht = Bal_sht(f"{dir}/PRE_balance_sheet.csv")

    def update(self):
        self.bal_sht.df.at['Net Income A', 'USD'] = self.inc_stat.df.at['Net Income', 'USD']
        self.bal_sht.df.at['Net Income E', 'USD'] = self.inc_stat.df.at['Net Income', 'USD']
        self.bal_sht.update()

    def transaction(self, prop, unit, category, cost, msrp, bill, date=datetime.datetime.now().date(), description=''):
        """
        :param prop: str - address
        :param cost: float - real cost
        :param msrp: float - market cost
        :param bill: str - entity to invoice
        :param category: str - ...
        :param description: str - ...
        :param unit: int
        :param date: datetime - date of transaction
        :rtype: none
        """
        transact = {'Date': date,
                  'Property': prop,
                  'Unit': unit,
                  'Category': category,
                  'Description': description,
                  'Cost': cost,
                  'MSRP': msrp,
                  'Bill': bill}
        
        self.ledger.df = self.ledger.df.append(transact, ignore_index=True)
        if transact['Category'] in self.inc_stat.df.index:
            self.inc_stat.entry(transact['Category'], transact['MSRP'])
            self.update()
        elif transact['Category'] in self.bal_sht.df.index:
            self.bal_sht.entry(transact['Category'], transact['MSRP'])
        else:
            raise NameError("Unknown category!")
    
    def commit(self):
        self.ledger.df = self.ledger.df.append(self.transaction(), ignore_index=True)

fs = FinStat("/content/")
fs.transaction('WF', 4, 'Gross Income', 100.00, 120.00, 'BREI', date=datetime.datetime(2022,1,11).date(), description='WF1 Rent')
fs.bal_sht.df

,USD
Net Income A,120.0
Cash,0.0
Account Receivable,0.0
Securities,0.0
Property,0.0
Total Asset,120.0
Account Payable,0.0
Unearned Revenue,0.0
Debt,0.0
Total Liability,0.0
